In [ ]:
import ee
import geemap

Map = geemap.Map()
Map

In [ ]:
import os
os.chdir(r'D:\Work\ERA5')

In [ ]:
start_time = 2000
end_time = 2010
crs_transform = [0.1, 0, -180.05, 0, -0.1, 90.05]
crs = 'epsg:4326'
shape = [1801, 3601]
bands = ['temperature_2m',
        'total_precipitation']
types = ['TAVG', 'PRCP']

In [ ]:
images = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{start_time}', f'{end_time}')

In [ ]:
images_id = images.aggregate_array('system:id').getInfo()

In [ ]:
for t in types:
    if not os.path.exists(t):
        os.mkdir(t)

In [ ]:
for img_id in images_id:
    # img_id = images_id[0]
    image = ee.Image(img_id)
    tavg = image.select(bands[0])
    prcp = image.select(bands[1])
    tavg_outfile = f"{types[0]}/{types[0]}_{img_id.split('/')[-1]}.tif"
    geemap.download_ee_image(tavg, tavg_outfile, crs=crs, crs_transform=crs_transform, shape=shape)
    prcp_outfile = f"{types[1]}/{types[1]}_{img_id.split('/')[-1]}.tif"
    geemap.download_ee_image(prcp, prcp_outfile, crs=crs, crs_transform=crs_transform, shape=shape)
    break

#### RHU 8day download

In [ ]:
var = 'RHU'
if not os.path.exists(var):
    os.mkdir(var)
    
for year in range(start_time, end_time):
    if (year%4 == 0 and year%400!=0) or (year%400==0):
        days = 366
    else:
        days = 365
    images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
    
    imgs_day_to_year = []
    for day in range(1, days+1):
        images_day = images_year.filter(ee.Filter.calendarRange(day, day, 'day_of_year'))

        tmax = images_day.select(bands[0]).max()
        tmin = images_day.select(bands[0]).min()
        tdew = images_day.select(bands[1]).mean()
        img = ee.Image([tmax, tmin, tdew])
        Ea = img.expression('0.6108 * exp(17.27 * b(2) / (237.3 + b(2)))')
        Etmax = img.expression('0.6108 * exp(17.27 * b(0) / (237.3 + b(0)))')
        Etmin = img.expression('0.6108 * exp(17.27 * b(1) / (237.3 + b(1)))')
        Es = Etmax.add(Etmin).divide(2)
        img_day = Ea.divide(Es).multiply(100)
        
        imgs_day_to_year.append(img_day)

    for i in range(0, days, day_sep):
        image = ee.ImageCollection(imgs_day_to_year[i:i+day_sep]).mean()
        outfile = f'{var}/{var}_{year}{i+1:03d}.tif'
        while not os.path.exists(outfile):
            try:
                download_image(image, outfile, region=roi, crs=crs,
                                     scale=scale)
            except:
                continue

#### SNR 8day download

In [ ]:
var = 'SNR'
if not os.path.exists(var):
    os.mkdir(var)
    
for year in range(start_time, end_time):
    if (year%4 == 0 and year%400!=0) or (year%400==0):
        days = 366
    else:
        days = 365
    images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
    
    imgs_day_to_year = []
    for day in range(days):
        images_day = images_year.filter(ee.Filter.calendarRange(day, day+1, 'day_of_year'))

        sntr = images_day.select(bands[3]).sum()
        snsr = images_day.select(bands[4]).sum()
        img_day = sntr.add(snsr)
        
        imgs_day_to_year.append(img_day)

    for i in range(0, days, day_sep):
        image = ee.ImageCollection(imgs_day_to_year[i:i+day_sep]).mean().divide(1000000)
        outfile = f'{var}/{var}_{year}{i+1:03d}.tif'
        while not os.path.exists(outfile):
            try:
                download_image(image, outfile, region=roi, crs=crs,
                                     scale=scale)
            except:
                continue


#### SSRD 8day download

In [ ]:
var = 'SSRD'
if not os.path.exists(var):
    os.mkdir(var)
    
for year in range(start_time, end_time):
    if (year%4 == 0 and year%400!=0) or (year%400==0):
        days = 366
    else:
        days = 365
    images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
    
    imgs_day_to_year = []
    for day in range(days):
        images_day = images_year.filter(ee.Filter.calendarRange(day, day+1, 'day_of_year'))

        img_day = images_day.select(bands[5]).sum()
        
        imgs_day_to_year.append(img_day)

    for i in range(0, days, day_sep):
        image = ee.ImageCollection(imgs_day_to_year[i:i+day_sep]).mean().divide(1000000)
        outfile = f'{var}/{var}_{year}{i+1:03d}.tif'
        while not os.path.exists(outfile):
            try:
                download_image(image, outfile, region=roi, crs=crs,
                                     scale=scale)
            except:
                continue